In [9]:
import mysql.connector
from getpass import getpass
import pandas as pd

In [43]:
#1. Establish a connection between Python and the Sakila database.
password = getpass()
cnx = mysql.connector.connect(user = "root",
                              password = password,
                              host = "localhost",
                              database = "sakila")

cnx.is_connected()

········


True

In [44]:
cursor = cnx.cursor()

In [47]:
# 2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

# 	- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
# 	- `month`: an integer representing the month for which rental data is to be retrieved.
# 	- `year`: an integer representing the year for which rental data is to be retrieved.

def rentals_month(cnx,month, year):
    query = ("""SELECT YEAR(rental_date) AS rental_year, 
            MONTH(rental_date) AS rental_month, 
            customer_id
            FROM sakila.rental;""") 
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.description
    columns = [header[0] for header in cursor.description]
    rental_df = pd.DataFrame(results, columns = columns)

    return rental_df

In [48]:
cnx.is_connected()

may_rentals = rentals_month(cnx, month=5, year=2005)
june_rentals = rentals_month(cnx, month=6, year=2005)
print(may_rentals)


       rental_year  rental_month  customer_id
0             2005             5          130
1             2005             5          459
2             2005             5          408
3             2005             5          333
4             2005             5          222
...            ...           ...          ...
16039         2006             2          274
16040         2006             2          374
16041         2006             2          216
16042         2006             2          532
16043         2006             2          373

[16044 rows x 3 columns]


In [49]:
# Identify unique customers who rented movies in both May and June
may_customers = set(may_rentals['customer_id'])
june_customers = set(june_rentals['customer_id'])
common_customers = may_customers.intersection(june_customers)

In [55]:
may_common_customers_rentals = may_rentals[may_rentals['customer_id'].isin(common_customers)]
june_common_customers_rentals = june_rentals[june_rentals['customer_id'].isin(common_customers)]

# Compare rental activity between May and June for common customers
common_customers_comparison = pd.merge(
    may_common_customers_rentals,
    june_common_customers_rentals,
    on='customer_id',
    suffixes=('_may', '_june')
)

common_customers_comparison

,rental_year_may,rental_month_may,customer_id,rental_year_june,rental_month_june
0,2005,5,130,2005,5
1,2005,5,130,2005,5
2,2005,5,130,2005,6
3,2005,5,130,2005,6
4,2005,5,130,2005,6
...,...,...,...,...,...
445345,2005,8,195,2005,8
445346,2005,8,195,2005,8
445347,2005,8,195,2005,8
445348,2005,8,195,2005,8


In [64]:
#3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 
def rental_count_month(rentals_df, month, year):
    # Extract year and month from the rental_date
    rentals_df['rental_year'] = pd.DatetimeIndex(rentals_df['rental_year']).year
    rentals_df['rental_month'] = pd.DatetimeIndex(rentals_df['rental_month']).month

    # Filter data for the specified month and year
    filtered_rentals = rentals_df[(rentals_df['rental_year'] == year) & (rentals_df['rental_month'] == month)]

    # Group by customer_id and count the number of rentals
    rental_counts = filtered_rentals.groupby('customer_id').size().reset_index(name=f'rentals_{month}_{year}')

    return rental_counts

In [76]:
rental_counts_may_2005 = rental_count_month(may_rentals, 5, 2005)
rental_counts_june_2005 = rental_count_month(june_rentals, 6, 2005)

In [77]:
rental_counts_june_2005

,customer_id,rentals_6_2005


In [78]:
def compare_rentals(rentals_1, rentals_2):
    # Merge DataFrames on 'customer_id'
    merged_rentals = pd.merge(rentals_1, rentals_2, on='customer_id', how='outer', suffixes=('_month1', '_month2'))

    # Fill NaN values with 0, as NaN indicates no rentals for that customer in a particular month
    merged_rentals = merged_rentals.fillna(0)

    # Calculate the difference between the number of rentals in the two months
    merged_rentals['difference'] = merged_rentals['rentals_month1'] - merged_rentals['rentals_month2']

    return merged_rentals

In [79]:
comparison_result = compare_rentals(rental_counts_may_2005, rental_counts_june_2005)

KeyError: 'rentals_month1'